In [1]:
import xarray as xr
import numpy as np
import torch
import matplotlib.pyplot as plt
import sys

path = '/home/akazemi3/Desktop/MB_Lab_Project/'
sys.path.append(path)
from analysis.neural_data_regression.tools.regression import *

from tools.processing import *
from tools.loading import *

import pandas as pd
import seaborn as sns
import pickle

BEST_ALPHA_PATH = '/data/atlas/regression_alphas'    
ROOT_PATH = '/data/atlas/model_scores'

In [2]:
def get_best_alpha(data_dict,legend_name_dict):
    
    df = pd.DataFrame()

    index = 0
    for model_name, model_info in data_dict.items():
                
        for alpha in model_info['alphas']:

            regression = f'Ridge(alpha={alpha})' 
                

            data = xr.open_dataset(os.path.join(ROOT_PATH,f'{model_name}_{regression}'))
            r_values = data.where(data.region == region,drop=True).r_value.values
            mean_r = np.mean(r_values)
            df_tmp =  pd.DataFrame({'mean_score':mean_r,
                                    'alpha':alpha,
                                    'model':model_name},index=[index])
            df = pd.concat([df,df_tmp])
            index += 1
    
    if legend_name_dict is not None:
        df.model = df.model.map(legend_name_dict)
    
    df = df.groupby(['model','alpha']).mean().reset_index()
    max_idx = df.groupby(['model']).agg({'mean_score':('idxmax')}).reset_index()['mean_score'].tolist()
    df_max_alphas = df.loc[max_idx].reset_index(drop=True)

    return df_max_alphas
    

In [3]:
dataset = 'naturalscenes_zscored_processed'
REGIONS = ['V4']
# dataset = 'majajhong'
# REGIONS = ['V4','IT']
mode = 'train'
#alpha_file_name = 'model_3L_mp'
alpha_file_name = 'model_3L_PCA'


                
for region in REGIONS:
    data_dict = {

           f'model_3L_PCA_more_pcs_{dataset}_{region}_{mode}':{'alphas': [10**i for i in range(2,5)]},
        
        
           #f'model_3L_mp_10000_avgpool_{dataset}_{region}_{mode}':{'alphas': [10**i for i in range(1,6)]},
           #  f'model_3L_mp_100_{dataset}_{region}_{mode}':{'alphas': [10**i for i in range(6)]},
           # f'model_3L_mp_1000_{dataset}_{region}_{mode}':{'alphas': [10**i for i in range(6)]},
           #f'model_3L_10000_{dataset}_{region}_{mode}':{'alphas': [10**i for i in range(6)]},
           # f'model_3L_mp_100000_{dataset}_{region}_{mode}':{'alphas': [10**i for i in range(2,5)]},
        
            # f'alexnet_untrained_mp_{dataset}_{region}_{mode}':{'alphas': [10**i for i in range(6)]},
            #f'alexnet_mp_{dataset}_{region}_{mode}':{'alphas': [10**i for i in range(6)]},
        
        
        #  f'model_3L_mp_10000_nsd_pca_{dataset}_10_{region}_{mode}':{'alphas': [10**i for i in range(1,5)]},
           #  f'model_3L_mp_10000_nsd_pca_{dataset}_100_{region}_{mode}':{'alphas': [10**i for i in range(1,5)]},
            # f'model_3L_mp_10000_nsd_pca_{dataset}_256_{region}_{mode}':{'alphas': [10**i for i in range(1,5)]},
           #  f'model_3L_mp_10000_nsd_pca_{dataset}_1000_{region}_{mode}':{'alphas': [10**i for i in range(1,5)]},
           #  f'model_3L_mp_10000_nsd_pca_{dataset}_5000_{region}_{mode}':{'alphas': [10**i for i in range(1,5)]},
           # f'model_3L_mp_10000_nsd_pca_{dataset}_10000_{region}_{mode}':{'alphas': [10**i for i in range(1,4)]},

        
#             f'model_3L_mp_50000_nsd_pca_{dataset}_10_{region}_{mode}':{'alphas': [10**i for i in range(1,5)]},
#             f'model_3L_mp_50000_nsd_pca_{dataset}_100_{region}_{mode}':{'alphas': [10**i for i in range(1,5)]},
#             f'model_3L_mp_50000_nsd_pca_{dataset}_256_{region}_{mode}':{'alphas': [10**i for i in range(1,5)]},
#             f'model_3L_mp_50000_nsd_pca_{dataset}_1000_{region}_{mode}':{'alphas': [10**i for i in range(1,5)]},
#             f'model_3L_mp_50000_nsd_pca_{dataset}_5000_{region}_{mode}':{'alphas': [10**i for i in range(1,5)]},
#            f'model_3L_mp_50000_nsd_pca_{dataset}_10000_{region}_{mode}':{'alphas': [10**i for i in range(1,5)]},
        
            # f'alexnet_untrained_wide_mp_10000_nsd_pca_{dataset}_10_{region}_{mode}':{'alphas': [10**i for i in range(1,5)]},
            # f'alexnet_untrained_wide_mp_10000_nsd_pca_{dataset}_100_{region}_{mode}':{'alphas': [10**i for i in range(1,5)]},
            # f'alexnet_untrained_wide_mp_10000_nsd_pca_{dataset}_256_{region}_{mode}':{'alphas': [10**i for i in range(1,5)]},
            # f'alexnet_untrained_wide_mp_10000_nsd_pca_{dataset}_1000_{region}_{mode}':{'alphas': [10**i for i in range(1,5)]},
            # f'alexnet_untrained_wide_mp_10000_nsd_pca_{dataset}_5000_{region}_{mode}':{'alphas': [10**i for i in range(1,5)]},
            # f'alexnet_untrained_wide_mp_10000_nsd_pca_{dataset}_10000_{region}_{mode}':{'alphas': [10**i for i in range(1,4)]},

#             f'alexnet_mp_nsd_pca_{dataset}_10_{region}_{mode}':{'alphas': [10**i for i in range(1,5)]},
#             f'alexnet_mp_nsd_pca_{dataset}_100_{region}_{mode}':{'alphas': [10**i for i in range(1,5)]},
#             f'alexnet_mp_nsd_pca_{dataset}_256_{region}_{mode}':{'alphas': [10**i for i in range(1,5)]},
    }

    legend_name_dict = {
            
            f'model_3L_PCA_more_pcs_{dataset}_{region}_{mode}':f'model_3L_PCA_more_pcs',
        
           #f'model_3L_mp_10000_avgpool_{dataset}_{region}_{mode}':f'model_3L_mp_10000_avgpool',
            # f'model_3L_mp_100_{dataset}_{region}_{mode}':f'model_3L_mp_100',
            # f'model_3L_mp_1000_{dataset}_{region}_{mode}':f'model_3L_mp_1000',
            #f'model_3L_10000_{dataset}_{region}_{mode}':f'model_3L_10000',
            # f'model_3L_mp_100000_{dataset}_{region}_{mode}':f'model_3L_mp_100000',
            # f'alexnet_untrained_mp_{dataset}_{region}_{mode}':'alexnet_untrained_mp',
            #f'alexnet_mp_{dataset}_{region}_{mode}':'alexnet_mp',

        
        
        
            #  f'model_3L_mp_10000_nsd_pca_{dataset}_10_{region}_{mode}':f'model_3L_mp_10000_nsd_pca_{dataset}_10',
           #  f'model_3L_mp_10000_nsd_pca_{dataset}_100_{region}_{mode}':f'model_3L_mp_10000_nsd_pca_{dataset}_100',
            # f'model_3L_mp_10000_nsd_pca_{dataset}_256_{region}_{mode}':f'model_3L_mp_10000_nsd_pca_{dataset}_256',
           #  f'model_3L_mp_10000_nsd_pca_{dataset}_1000_{region}_{mode}':f'model_3L_mp_10000_nsd_pca_{dataset}_1000',
           #  f'model_3L_mp_10000_nsd_pca_{dataset}_5000_{region}_{mode}':f'model_3L_mp_10000_nsd_pca_{dataset}_5000',
           # f'model_3L_mp_10000_nsd_pca_{dataset}_10000_{region}_{mode}':f'model_3L_mp_10000_nsd_pca_{dataset}_10000',

#             f'model_3L_mp_50000_nsd_pca_{dataset}_10_{region}_{mode}':f'model_3L_mp_50000_nsd_pca_{dataset}_10',
#             f'model_3L_mp_50000_nsd_pca_{dataset}_100_{region}_{mode}':f'model_3L_mp_50000_nsd_pca_{dataset}_100',
#             f'model_3L_mp_50000_nsd_pca_{dataset}_256_{region}_{mode}':f'model_3L_mp_50000_nsd_pca_{dataset}_256',
#             f'model_3L_mp_50000_nsd_pca_{dataset}_1000_{region}_{mode}':f'model_3L_mp_50000_nsd_pca_{dataset}_1000',
#             f'model_3L_mp_50000_nsd_pca_{dataset}_5000_{region}_{mode}':f'model_3L_mp_50000_nsd_pca_{dataset}_5000',
#            f'model_3L_mp_50000_nsd_pca_{dataset}_10000_{region}_{mode}':f'model_3L_mp_50000_nsd_pca_{dataset}_10000',
        
            # f'alexnet_untrained_wide_mp_10000_nsd_pca_{dataset}_10_{region}_{mode}':f'alexnet_untrained_wide_mp_10000_nsd_pca_{dataset}_10',
            # f'alexnet_untrained_wide_mp_10000_nsd_pca_{dataset}_100_{region}_{mode}': f'alexnet_untrained_wide_mp_10000_nsd_pca_{dataset}_100',
            # f'alexnet_untrained_wide_mp_10000_nsd_pca_{dataset}_256_{region}_{mode}':f'alexnet_untrained_wide_mp_10000_nsd_pca_{dataset}_256',
            # f'alexnet_untrained_wide_mp_10000_nsd_pca_{dataset}_1000_{region}_{mode}':f'alexnet_untrained_wide_mp_10000_nsd_pca_{dataset}_1000',
            # f'alexnet_untrained_wide_mp_10000_nsd_pca_{dataset}_5000_{region}_{mode}':f'alexnet_untrained_wide_mp_10000_nsd_pca_{dataset}_5000',
            # f'alexnet_untrained_wide_mp_10000_nsd_pca_{dataset}_10000_{region}_{mode}':f'alexnet_untrained_wide_mp_10000_nsd_pca_{dataset}_10000',

#             #  f'alexnet_mp_nsd_pca_{dataset}_10_{region}_{mode}':f'alexnet_mp_nsd_pca_{dataset}_10',
            # f'alexnet_mp_nsd_pca_{dataset}_100_{region}_{mode}':f'alexnet_mp_nsd_pca_{dataset}_100',
            # f'alexnet_mp_nsd_pca_{dataset}_256_{region}_{mode}':f'alexnet_mp_nsd_pca_{dataset}_256',
    }

    df_best_alpha = get_best_alpha(data_dict,legend_name_dict)
    print(region)
    print(df_best_alpha)
    # dict_best_alpha = df_best_alpha.set_index(['model']).to_dict()['alpha']
    # file = open(os.path.join(BEST_ALPHA_PATH, f'{alpha_file_name}_{dataset}_{region}'),'wb')
    # pickle.dump(dict_best_alpha, file)
    # file.close()

FileNotFoundError: [Errno 2] No such file or directory: b'/data/atlas/model_scores/model_3L_PCA_more_pcs_naturalscenes_zscored_processed_V4_train_Ridge(alpha=100)'

In [12]:
xr.open_dataset('/data/atlas/model_scores/model_3L_PCA_naturalscenes_zscored_processed_V4_test_Ridge(alpha=1000)').r_value.values.mean()

0.19802386560046387

In [ ]:
xr.open_dataset('/data/atlas/model_scores/model_3L_PCA_naturalscenes_zscored_processed_V4_test_Ridge(alpha=1000)').r_value.values.mean()